<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Interactive-Visualization-Lab" data-toc-modified-id="Interactive-Visualization-Lab-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Interactive Visualization Lab</a></span><ul class="toc-item"><li><span><a href="#1.-Create-an-interactive-bar-chart-showing-total-quantity-and-revenue-by-country-(excluding-United-Kingdom)-for-the-month-of-April-2011." data-toc-modified-id="1.-Create-an-interactive-bar-chart-showing-total-quantity-and-revenue-by-country-(excluding-United-Kingdom)-for-the-month-of-April-2011.-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.</a></span></li><li><span><a href="#2.-Create-an-interactive-line-chart-showing-quantity-and-revenue-sold-to-France-between-January-1st-and-May-31st-2011." data-toc-modified-id="2.-Create-an-interactive-line-chart-showing-quantity-and-revenue-sold-to-France-between-January-1st-and-May-31st-2011.-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.</a></span></li><li><span><a href="#3.-Create-an-interactive-scatter-plot-showing-the-relationship-between-average-quantity-(x-axis)-and-average-unit-price-(y-axis)-for-the-product-PARTY-BUNTING-with-the-plot-points-color-coded-by-country-(categories)." data-toc-modified-id="3.-Create-an-interactive-scatter-plot-showing-the-relationship-between-average-quantity-(x-axis)-and-average-unit-price-(y-axis)-for-the-product-PARTY-BUNTING-with-the-plot-points-color-coded-by-country-(categories).-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).</a></span></li><li><span><a href="#4.-Create-a-set-of-interactive-histograms-showing-the-distributions-of-quantity-per-invoice-for-the-following-countries:-EIRE,-Germany,-France,-and-Netherlands." data-toc-modified-id="4.-Create-a-set-of-interactive-histograms-showing-the-distributions-of-quantity-per-invoice-for-the-following-countries:-EIRE,-Germany,-France,-and-Netherlands.-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.</a></span></li><li><span><a href="#5.-Create-an-interactive-side-by-side-bar-chart-showing-the-revenue-by-country-listed-below-(bars)-for-each-of-the-products-listed-below." data-toc-modified-id="5.-Create-an-interactive-side-by-side-bar-chart-showing-the-revenue-by-country-listed-below-(bars)-for-each-of-the-products-listed-below.-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.</a></span></li><li><span><a href="#6.-Create-an-interactive-line-chart-showing-quantity-sold-by-day-for-the-United-Kingdom.-Add-drop-down-boxes-for-Year-and-Month-that-allow-you-to-filter-the-date-range-that-appears-in-the-chart." data-toc-modified-id="6.-Create-an-interactive-line-chart-showing-quantity-sold-by-day-for-the-United-Kingdom.-Add-drop-down-boxes-for-Year-and-Month-that-allow-you-to-filter-the-date-range-that-appears-in-the-chart.-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.</a></span></li><li><span><a href="#7.-Create-an-interactive-scatter-plot-that-plots-number-of-invoices-(x-axis)-vs.-number-of-customers-(y-axis)-and-the-plot-points-represent-individual-products.-Add-two-sliders-that-control-the-x-and-y-axis-ranges." data-toc-modified-id="7.-Create-an-interactive-scatter-plot-that-plots-number-of-invoices-(x-axis)-vs.-number-of-customers-(y-axis)-and-the-plot-points-represent-individual-products.-Add-two-sliders-that-control-the-x-and-y-axis-ranges.-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.</a></span></li><li><span><a href="#8.-Creat-an-interactive-bar-chart-that-shows-revenue-by-product-description.-Add-a-text-field-widget-that-filters-the-results-to-show-the-product-that-contain-the-text-entered-in-their-description." data-toc-modified-id="8.-Creat-an-interactive-bar-chart-that-shows-revenue-by-product-description.-Add-a-text-field-widget-that-filters-the-results-to-show-the-product-that-contain-the-text-entered-in-their-description.-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.</a></span></li></ul></li></ul></div>

# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [1]:
import pandas as pd
import plotly.plotly as py
import cufflinks as cf
from ipywidgets import interact

cf.go_offline()

In [8]:
data = pd.read_excel('../data/Online Retail.xlsx')

## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [ ]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [ ]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [ ]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func)

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.